<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_DEEPSEEK_ONCOLOGY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import json
from openai import OpenAI
from google.colab import userdata

# --- API Configuration ---
api_key = userdata.get('DEEPSEEK_API_KEY')
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")

# --- Tool Definitions ---
# These are the functions that the Executor Agent can call.
def get_patient_ehr(patient_id: str) -> dict:
    """Simulates retrieving a patient's Electronic Health Record for an oncology case."""
    ehr_data = {
        "12345": {
            "name": "Jane Doe",
            "age": 55,
            "gender": "Female",
            "past_history": "History of breast cancer (T1N0M0, s/p lumpectomy and radiation), family history of breast and ovarian cancer.",
            "allergies": "Penicillin",
            "current_medications": ["Tamoxifen"],
            "labs": {"CA-125": "elevated"}
        }
    }
    return ehr_data.get(patient_id, {"error": "Patient not found"})

def order_biopsy(patient_id: str, reason: str) -> str:
    """Simulates ordering a biopsy for a patient."""
    return f"Biopsy order for patient {patient_id} placed. Reason: {reason}. Results will be available shortly."

def get_biopsy_results(patient_id: str) -> dict:
    """Simulates getting biopsy results for a patient."""
    return {"patient_id": patient_id, "findings": "Invasive ductal carcinoma (IDC), HER2-positive."}

def order_ct_scan(patient_id: str, reason: str) -> str:
    """Simulates ordering a CT scan for a patient."""
    return f"CT scan order for patient {patient_id} placed. Reason: {reason}. Results will be available shortly."

def get_ct_scan_results(patient_id: str) -> dict:
    """Simulates getting CT scan results for a patient."""
    return {"patient_id": patient_id, "findings": "No evidence of distant metastasis."}

def get_tumor_marker_results(patient_id: str) -> dict:
    """Simulates getting tumor marker results."""
    return {"patient_id": patient_id, "findings": "CA-125 level is 150 U/mL (Elevated)."}

def diagnose_and_recommend_oncology_plan(patient_id: str, ehr_data: dict, biopsy_results: dict, ct_scan_results: dict, tumor_markers: dict) -> dict:
    """Acts as a Specialist Oncologist Agent to synthesize all diagnostic data and provide a comprehensive plan."""
    summary = f"Patient {ehr_data['name']}, a {ehr_data['age']}-year-old {ehr_data['gender']} with a history of breast cancer and family history of breast/ovarian cancer, has been evaluated for potential recurrence."

    findings = f"Diagnostic findings include: \
    \n- Biopsy: {biopsy_results['findings']} \
    \n- CT Scan: {ct_scan_results['findings']} \
    \n- Tumor Markers: {tumor_markers['findings']}"

    recommendation = "Based on these findings, I recommend:"

    # Example of a decision based on findings
    if "Elevated" in tumor_markers['findings'] and "no evidence of distant metastasis" in ct_scan_results['findings']:
        plan = "1. Immediate consultation with a surgical oncologist to discuss biopsy findings.\
        \n2. Further imaging, such as a PET scan or MRI, to look for micro-metastases not seen on CT.\
        \n3. Start targeted therapy with an anti-HER2 agent (e.g., Herceptin) given the HER2-positive status.\
        \n4. Consider genetic counseling and testing due to the family history."
    else:
        plan = "1. Review all findings in a multidisciplinary tumor board.\
        \n2. Follow up with the patient to discuss treatment options."

    return {
        "summary": summary,
        "findings": findings,
        "recommendation": recommendation,
        "plan": plan
    }

# Corrected tool definitions with proper JSON schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_patient_ehr",
            "description": "Retrieves detailed Electronic Health Record for a patient, including oncology-specific history and labs.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_biopsy",
            "description": "Places an order for a biopsy.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "reason": {"type": "string"}
                }, "required": ["patient_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_biopsy_results",
            "description": "Retrieves the biopsy results for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "order_ct_scan",
            "description": "Places an order for a CT scan.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "reason": {"type": "string"}
                }, "required": ["patient_id", "reason"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_ct_scan_results",
            "description": "Retrieves the CT scan results for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_tumor_marker_results",
            "description": "Retrieves the tumor marker results for a patient.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"}
                }, "required": ["patient_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "diagnose_and_recommend_oncology_plan",
            "description": "Synthesizes all diagnostic data to provide a comprehensive oncology plan from an expert's perspective.",
            "parameters": {
                "type": "object",
                "properties": {
                    "patient_id": {"type": "string"},
                    "ehr_data": {"type": "object"},
                    "biopsy_results": {"type": "object"},
                    "ct_scan_results": {"type": "object"},
                    "tumor_markers": {"type": "object"}
                },
                "required": ["patient_id", "ehr_data", "biopsy_results", "ct_scan_results", "tumor_markers"]
            }
        }
    }
]

# --- Main Orchestration Logic ---
def run_agentic_flow_corrected(user_prompt: str, patient_id: str):
    """
    Main function to orchestrate the multi-agent workflow for an oncology case.
    """
    try:
        messages = [
            {"role": "system", "content": "You are a clinical decision support agent specializing in oncology. Use your tools to gather all necessary diagnostic information before calling the specialist oncology agent to provide the final, synthesized recommendation to the user."},
            {"role": "user", "content": f"{user_prompt}. Patient ID: {patient_id}"}
        ]

        patient_info = {}

        for turn in range(10): # Increased turns for more complex planning
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=tools,
                tool_choice="auto"
            )

            response_message = response.choices[0].message
            messages.append(response_message)

            if not response_message.tool_calls:
                return response_message.content

            print("\nModel has decided to call tools...")

            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                arguments = json.loads(tool_call.function.arguments)
                tool_call_id = tool_call.id

                print(f"Executing tool: {function_name} with arguments: {arguments}...")

                if function_name == "get_patient_ehr":
                    tool_response_content = get_patient_ehr(**arguments)
                    patient_info['ehr'] = tool_response_content
                elif function_name == "order_biopsy":
                    tool_response_content = order_biopsy(**arguments)
                elif function_name == "get_biopsy_results":
                    tool_response_content = get_biopsy_results(**arguments)
                    patient_info['biopsy'] = tool_response_content
                elif function_name == "order_ct_scan":
                    tool_response_content = order_ct_scan(**arguments)
                elif function_name == "get_ct_scan_results":
                    tool_response_content = get_ct_scan_results(**arguments)
                    patient_info['ct_scan'] = tool_response_content
                elif function_name == "get_tumor_marker_results":
                    tool_response_content = get_tumor_marker_results(**arguments)
                    patient_info['tumor_markers'] = tool_response_content
                elif function_name == "diagnose_and_recommend_oncology_plan":
                    # Call the new specialist agent with the collected data
                    tool_response_content = diagnose_and_recommend_oncology_plan(
                        patient_id=arguments.get('patient_id'),
                        ehr_data=patient_info.get('ehr'),
                        biopsy_results=patient_info.get('biopsy'),
                        ct_scan_results=patient_info.get('ct_scan'),
                        tumor_markers=patient_info.get('tumor_markers')
                    )
                else:
                    tool_response_content = {"error": "Unknown tool."}

                messages.append(
                    {
                        "tool_call_id": tool_call_id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(tool_response_content)
                    }
                )
        return "Max conversation turns reached without a final answer."

    except Exception as e:
        return f"An error occurred: {e}"

# --- Execution ---
if __name__ == "__main__":
    patient_query = "A 55-year-old female with a history of breast cancer and elevated tumor markers needs evaluation for potential recurrence."
    patient_id = "12345"

    print("--- Agentic Clinical Decision Support System (Oncology) ---")
    print(f"User Query: {patient_query}\n")

    result = run_agentic_flow_corrected(user_prompt=patient_query, patient_id=patient_id)
    print("\n--- Final Agent Recommendation ---")
    print(result)


--- Agentic Clinical Decision Support System (Oncology) ---
User Query: A 55-year-old female with a history of breast cancer and elevated tumor markers needs evaluation for potential recurrence.


Model has decided to call tools...
Executing tool: get_patient_ehr with arguments: {'patient_id': '12345'}...

Model has decided to call tools...
Executing tool: get_tumor_marker_results with arguments: {'patient_id': '12345'}...

Model has decided to call tools...
Executing tool: order_ct_scan with arguments: {'patient_id': '12345', 'reason': 'Evaluation for breast cancer recurrence with elevated CA-125 tumor marker'}...

Model has decided to call tools...
Executing tool: get_ct_scan_results with arguments: {'patient_id': '12345'}...

Model has decided to call tools...
Executing tool: order_biopsy with arguments: {'patient_id': '12345', 'reason': 'Elevated CA-125 tumor marker with history of breast cancer, evaluating for recurrence or new primary malignancy'}...

Model has decided to call to